In [1]:
#train_path = '/data/home/dor/train'
#eval_path = '/data/home/dor/eval'
#model_path = '/data/home/dor/model'
train_path = 'train'
eval_path = 'eval'
model_path ='model'

In [2]:
from collections import deque
from gensim.models import Word2Vec
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import random

In [11]:
def conll_to_transitions(sentence):
    '''
    Given a sentence, returns a list of transitions.
    Each transition is a training instance for your classifier. A transition 
    is composed of the following 4 items:
    - first word in stack
    - second word in stack (could be None is stack is of size=1)
    - first word in buffer (could be None if the buffer is empty)
    - the transition label (SHIFT, LEFT, RIGHT)
    '''
    s = []  #stack
    b = deque([])  #buffer

    transitions = []

    for w in sentence:
        b.append(w)

    s.append(['0', 'ROOT', '_', '_', '_', '_', '0', '_', '_', '_'])

    while len(b) > 0 or len(s) > 1:
        if s[-1][0] == '0':   # the root
            add_shift(s, b, transitions)
        elif s[-2][6] == s[-1][0] and check_rightest_arc(s[-2], b):
            add_left(s, b, transitions)
        elif s[-1][6] == s[-2][0] and (len(b) == 0 or s[-2][0] != '0') and check_rightest_arc(s[-1], b):
            add_right(s, b, transitions)
        elif len(b) == 0:
            #print("Non projective")
            return None
        else:
            add_shift(s, b, transitions)
    return transitions


def check_rightest_arc(word, b):
    '''
   w[6] is the index of the head of "this" word, so in this method we check
   if there is an arc that goes from one of the words in the buffer
   to "word" (which exists in the stack)
    '''
    for w in b:
        if w[6] == word[0]:
            return False
    return True


def add_shift(s, b, transitions):
    '''
    Adding shift transition
    '''
    word = b.popleft()
    top2 = None
    if len(s) > 1:
        top2 = s[-2]
    transitions.append([s[-1], top2, word, 'SHIFT'])
    s.append(word)


def add_left(s, b, transitions):
    '''
    Adding left transition
    '''
    top1 = s.pop()
    top2 = s.pop()
    transitions.append([top1, top2, b[0] if len(b) > 0 else None, 'LEFT'])
    s.append(top1)


def add_right(s, b, transitions):
    '''
    Adding right transition
    '''
    top1 = s.pop()
    top2 = s.pop()
    transitions.append([top1, top2, b[0] if len(b) > 0 else None, 'RIGHT'])
    s.append(top2)

    
def transitions_to_conll(sentence, predicted_transitions, real_transitions):
    s = [0]
    b = deque([])
    result = {}
    for word in sentence:
        b.append(word)
    total = len(predicted_transitions)
    counter = 0
    while len(s) > 1 or len(b)>0:
        if len(predicted_transitions)>0:
            predicted_trans = predicted_transitions[0]
            real_trans = real_transitions[0][3]
            del predicted_transitions[0]
            del real_transitions[0]
        else:
            predicted_trans = None
        if predicted_trans == 'SHIFT' and not b:
            rand = random.uniform(0, 1)
            if (rand<=0.5):
                predicted_trans = 'LEFT'
            else:
                predicted_trans = 'RIGHT'
        if len(s) < 2 and b:
            predicted_trans = 'SHIFT'
        elif len(s) < 2:
            predicted_trans = 'RIGHT'

        if (predicted_trans == real_trans):
            counter += 1
        
        if predicted_trans == 'LEFT':
            t1 = s.pop()
            t2 = s.pop()
            result[t2]= t1
            s.append(t1)
        elif predicted_trans == 'SHIFT':
            s.append(b.popleft())
        elif predicted_trans == 'RIGHT':
            t1 = s.pop()
            t2 = s.pop()
            result[t1] = t2
            s.append(t2)
        
        
    return result, counter/total
            
            
    
    


In [12]:
def get_trans_list(input_file):
    h = open(input_file, 'r')
    sentence = []
    trans_list = []
    for l in h:
        l = l.strip()
        if l == "":
            trans = conll_to_transitions(sentence)
            trans_list.append(trans)
            #print("NEW")
            #print(trans)
            sentence = []
        else:
            sentence.append(l.split('\t'))
    h.close()
    return trans_list

train_trans_list = get_trans_list(train_path)
eval_trans_kist = get_trans_list(eval_path)
print(train_trans_list[1][1])

[['1', 'The', '_', 'DT', 'DT', '_', '4', 'NMOD', '_', '_'], ['0', 'ROOT', '_', '_', '_', '_', '0', '_', '_', '_'], ['2', 'current', '_', 'JJ', 'JJ', '_', '4', 'NMOD', '_', '_'], 'SHIFT']


# Creating word embbidings 

In [13]:
def get_sentences(files):
    ### return word2vec model using the train file
    sentences_list = []
    for file in files:
        h = open(file, 'r')
        sentence = ['ROOT']
        for l in h:
            l = l.strip()
            if l == "":
                sentences_list.append(sentence)
                sentence = ['ROOT']
            else:
                sentence.append(l.split('\t')[1])
        h.close()
    return sentences_list

In [14]:
sentences = get_sentences([train_path, eval_path])
word2vec_model = Word2Vec(sentences, window=7, min_count=1)
word2vec_model.save(model_path)

## Finding POS coarse version and POS tag more specific

In [15]:
def get_pos_set(input_file):
    h = open(input_file, 'r')
    POS_coarse_version = set()
    POS_more_specific = set()
    for l in h:
        l = l.strip()
        if l == "":
            continue
        else:
            l_split = l.split('\t')
            if l_split[3] != "_":
                POS_coarse_version.add(l_split[3])
            if l_split[4] != "_":
                POS_more_specific.add(l_split[4])
    h.close()
    return list(POS_coarse_version), list(POS_more_specific)

POS_coarse_version,POS_more_specific = get_pos_set(train_path)
print('POS_coarse_version ' + str(POS_coarse_version))
print('POS_more_specific ' + str(POS_more_specific))

POS_coarse_version ['NN', ')', '$', 'CD', '.', 'IN', 'UH', 'PR', 'TO', 'WD', 'PD', 'PO', '``', 'VB', "''", ',', 'WR', 'DT', 'WP', 'MD', 'CC', 'JJ', 'RB', 'FW', ':', 'EX', '(', 'RP']
POS_more_specific ['NN', 'VBP', ')', '$', 'CD', 'JJR', '.', 'IN', 'UH', 'TO', 'PRP', 'NNS', 'VBG', 'RBR', '``', 'WDT', 'VB', "''", ',', 'VBD', 'NNP', 'POS', 'DT', 'WP$', 'RBS', 'WP', 'MD', 'CC', 'JJS', 'JJ', 'RB', 'FW', 'WRB', ':', 'EX', 'PRP$', '(', 'VBZ', 'VBN', 'PDT', 'RP', 'NNPS']


# implementing approch 1

First, we create tran to vector using the following features:
* The two top words in the stack - presnted as word2vec
* The top word in the buffer - presnted as word2vec
* The POS of each word, as 1 vector - we use both coarse and more specific. We use 1 hot vector 

The following method return for each trans state: the training vector and the operation needed to be done in Nivre-Acr 
  standard {'SHIFT':0,'LEFT':1,'RIGHT':2}

In [16]:
def trans_to_train_vec_and_result(trans_row, POS_coarse_version, POS_more_specific, model):
    word2vec_size = model.vector_size
    POS_cv_len = len(POS_coarse_version)
    POS_ms_len = len(POS_more_specific)
    vector_size = 3*word2vec_size + 3*POS_cv_len + 3*POS_ms_len + 3
    i=0
    label_dict = {'SHIFT' : 0, 'LEFT' : '1', 'RIGHT': 2}
    x = np.zeros(vector_size)
    for row in trans_row:
        if i==3:
            y = label_dict[row]
            continue
        else:
            if (row != None):
                word = row[1]
                if (word == 'ROOT'):
                    word2vec = model.wv.get_vector(word)
                    POS_coarse_vec = np.zeros(POS_cv_len)
                    POS_more_specific_vec = np.zeros(POS_ms_len)
                else:
                    word2vec = model.wv.get_vector(word)
                    POS_coarse_vec = np.zeros(POS_cv_len)
                    np.put(POS_coarse_vec, POS_coarse_version.index(row[3]),1)
                    POS_more_specific_vec = np.zeros(POS_ms_len)
                    np.put(POS_more_specific_vec, POS_more_specific.index(row[4]),1)
            else:
                i = i + 1 
                continue
        x[i*word2vec_size : (i+1)*word2vec_size] = word2vec
        x[i*POS_cv_len + 3*word2vec_size : (i+1)*POS_cv_len + 3*word2vec_size] = POS_coarse_vec
        x[i*POS_ms_len + 3*word2vec_size + 3*POS_cv_len : (i+1)*POS_ms_len + 3*word2vec_size + 3*POS_cv_len ] = POS_more_specific_vec
        x[-3+i] =  int(row[0]) 
        
       
        i = i + 1 
    return x,y 
    

Let's statrt to train our ML model

In [17]:
word2vec_size = word2vec_model.vector_size
POS_cv_len = len(POS_coarse_version)
POS_ms_len = len(POS_more_specific)
vector_size = 3*word2vec_size + 3*POS_cv_len+3*POS_ms_len + 3
X = np.empty((50000, vector_size), float)
Y = np.empty((50000,1))
i = 0
for z in train_trans_list[1:]:
    if not z == None:
        for tri in z:
            x,y = trans_to_train_vec_and_result(tri, POS_coarse_version, POS_more_specific, word2vec_model)
            X[i] = x
            Y[i] = y
            i = i + 1
X = X[0:i-1, :]
Y = Y[0:i-1, :]

## evalute: 

Let's try KNN

In [92]:
print(Y.ravel().shape)

(44019,)


In [19]:
for k in range(3,13,2):
    knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    knn.fit(X, Y.ravel())
    index_dict = {0:'SHIFT',1:'LEFT',2:'RIGHT'}
    count = 0
    total = 0
    classifier_eval = 0
    sentence = []
    predicted_transiation = []
    h = open(eval_path, 'r')
    for l in h:
        l = l.strip()
        if l == "":
            real_transiation = conll_to_transitions(sentence)
            if real_transiation == None:
                continue
            for word in real_transiation:
                x,y = trans_to_train_vec_and_result(word, POS_coarse_version, POS_more_specific, word2vec_model)
                x = x.reshape(1, -1)
                predicted_trans = index_dict[int(knn.predict(x)[0])]
                predicted_transiation.append(predicted_trans)
            dictOfWords = { i+1 : sentence[i][1] for i in range(0, len(sentence) ) }  
            results, acc = transitions_to_conll(dictOfWords, predicted_transiation, real_transiation)
            
            for word in sentence:
                if  word[6] != "_":
                    if int(word[0]) in results and results[int(word[0])] == int(word[6]):
                          count = count + 1
            classifier_eval += acc * len(sentence)
            total = total + len(sentence)
            sentence = []
            svm_transition = []     
        else:
            sentence.append(l.split('\t'))
    h.close()
    print("Total accuracy of the classifier for k =" + str(k) + " is: " + str(classifier_eval/total))
    print("Total accuracy of matching word for k =" + str(k) + " is: " + str(count/total))

Total accuracy of the classifier for k =3 is: 0.7980910425844346
Total accuracy of matching word for k =3 is: 0.30983847283406757
Total accuracy of the classifier for k =5 is: 0.7856093979441997
Total accuracy of matching word for k =5 is: 0.2878120411160059
Total accuracy of the classifier for k =7 is: 0.7878120411160059
Total accuracy of matching word for k =7 is: 0.302496328928047
Total accuracy of the classifier for k =9 is: 0.7966226138032305
Total accuracy of matching word for k =9 is: 0.33480176211453744
Total accuracy of the classifier for k =11 is: 0.7929515418502202
Total accuracy of matching word for k =11 is: 0.3054331864904552


We can see that the classifier is right about 80% of the cases.
However, only 30% of the words are matched correctly because one error in classification can effect the following words.